In [1]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
import shutil
import csv
import re
from collections import defaultdict
from datetime import datetime
import pandas as pd
import numpy as np

import sys
sys.path.append("./missing_data")

from mpl_toolkits.basemap import Basemap
from dataset.build_dataset import get_files_from_folder, extract_dates_pattern_airmass_rgb_20200101_0000
from dataset.build_dataset import load_cyclones_track_noheader, compute_pixel_scale, inside_tile, calc_tile_offsets, save_single_tile
from dataset.build_dataset import split_into_tiles_subfolders_and_track_dates, create_and_save_tile_from_complete_df
from dataset.build_dataset import create_final_df_csv
from dataset.build_dataset import get_gruppi_date, create_tile_videos, group_df_by_offsets
from medicane_utils.geo_const import latcorners, loncorners, x_center, y_center, create_basemap_obj
from medicane_utils.load_files import load_all_images

basemap_obj = create_basemap_obj()

# Percorso alla cartella dei frame
input_dir = "../fromgcloud"
output_dir = "../airmassRGB/supervised/" 
unsup_output_dir = "../airmassRGB/unsupervised/" 
os.makedirs(unsup_output_dir, exist_ok=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/isac/miniconda3/envs/videomae/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Xmin, Ymin, px_scale_x, px_scale_y = compute_pixel_scale()
1/px_scale_y/1000 * 224, 1/px_scale_x/1000 * 224

#### load all file names and dates

In [2]:
#os.makedirs(output_dir, exist_ok=True)
sorted_metadata_files = load_all_images(input_dir)

117764 files loaded.


In [ ]:
f_sorted_metadata = sorted_metadata_files[:32]
#f_sorted_metadata

In [ ]:
num_total_files = len(f_sorted_metadata)
# quanti blocchi di 16 consecutivi
num_frames = 16
num_subfolders = num_total_files // num_frames

# Unsupervised dataset

In [14]:
nome_file = "all_data_unsup.csv"
df_data = pd.read_csv(nome_file, dtype={
        "path": 'string',
        "tile_offset_x": 'int16',
        "tile_offset_y": 'int16',
    }, parse_dates=['datetime'])
df_data.drop(columns="Unnamed: 0", inplace=True)

In [23]:
df_data = df_data[:1000]

In [24]:
df_data

,path,datetime,tile_offset_x,tile_offset_y
0,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01 00:00:00,0,0
1,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01 00:00:00,213,0
2,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01 00:00:00,426,0
3,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01 00:00:00,639,0
4,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01 00:00:00,852,0
...,...,...,...,...
995,../fromgcloud/airmass_rgb_20111101_0650.png,2011-11-01 06:50:00,1065,196
996,../fromgcloud/airmass_rgb_20111101_0655.png,2011-11-01 06:55:00,0,0
997,../fromgcloud/airmass_rgb_20111101_0655.png,2011-11-01 06:55:00,213,0
998,../fromgcloud/airmass_rgb_20111101_0655.png,2011-11-01 06:55:00,426,0


In [25]:
gruppi_date = get_gruppi_date(df_data)

In [26]:
len(gruppi_date)

1

In [ ]:
all_videos = []
for df in gruppi_date:
    df_offsets_groups = group_df_by_offsets(df)
    df_videos = create_tile_videos(df_offsets_groups, supervised=False)
    all_videos.append(df_videos)
    print(f"len(df_videos) {len(df_videos)}")
    create_and_save_tile_from_complete_df(df_videos, unsup_output_dir)

len(df_videos) 60
Salvati 0 file - Erano già presenti 960 file - File totali 960


In [28]:
all_df_videos = pd.concat(all_videos)

In [34]:
all_df_videos

,video_id,tile_offset_x,tile_offset_y,path,label,start_time,end_time,orig_paths
0,0,0,0,01-11-2011_0120_0_0,None,2011-11-01 00:00:00,2011-11-01 01:20:00,"[../fromgcloud/airmass_rgb_20111101_0000.png, ..."
1,1,0,0,01-11-2011_0240_0_0,None,2011-11-01 01:20:00,2011-11-01 02:40:00,"[../fromgcloud/airmass_rgb_20111101_0120.png, ..."
2,2,0,0,01-11-2011_0400_0_0,None,2011-11-01 02:40:00,2011-11-01 04:00:00,"[../fromgcloud/airmass_rgb_20111101_0240.png, ..."
3,3,0,0,01-11-2011_0520_0_0,None,2011-11-01 04:00:00,2011-11-01 05:20:00,"[../fromgcloud/airmass_rgb_20111101_0400.png, ..."
4,4,0,0,01-11-2011_0640_0_0,None,2011-11-01 05:20:00,2011-11-01 06:40:00,"[../fromgcloud/airmass_rgb_20111101_0520.png, ..."
5,5,0,196,01-11-2011_0120_0_196,None,2011-11-01 00:00:00,2011-11-01 01:20:00,"[../fromgcloud/airmass_rgb_20111101_0000.png, ..."
6,6,0,196,01-11-2011_0240_0_196,None,2011-11-01 01:20:00,2011-11-01 02:40:00,"[../fromgcloud/airmass_rgb_20111101_0120.png, ..."
7,7,0,196,01-11-2011_0400_0_196,None,2011-11-01 02:40:00,2011-11-01 04:00:00,"[../fromgcloud/airmass_rgb_20111101_0240.png, ..."
8,8,0,196,01-11-2011_0520_0_196,None,2011-11-01 04:00:00,2011-11-01 05:20:00,"[../fromgcloud/airmass_rgb_20111101_0400.png, ..."
9,9,0,196,01-11-2011_0640_0_196,None,2011-11-01 05:20:00,2011-11-01 06:40:00,"[../fromgcloud/airmass_rgb_20111101_0520.png, ..."


In [32]:
df_dataset_csv_unsup = create_final_df_csv(df_videos, unsup_output_dir)

/media/isacDisk1/VideoMAEv2/build_dataset.py:447: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dataset_csv['path'] = output_dir + df_dataset_csv['path']
/media/isacDisk1/VideoMAEv2/build_dataset.py:448: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dataset_csv['start'] = 1
/media/isacDisk1/VideoMAEv2/build_dataset.py:449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [33]:
df_dataset_csv_unsup.drop(columns='label').to_csv("./train_960_UNsupervised.csv", index=False)

# Supervised

### Dataframe di tutte le tiles etichettate - check

In [13]:
nome_file = "all_data_full_tiles.csv"
#nome_file = "all_data_wo_overlap_tiles.csv"
df_data = pd.read_csv(nome_file, dtype={
        "path": 'string',
        "tile_offset_x": 'int16',
        "tile_offset_y": 'int16',
        "label": 'int16',
        "lat": 'object',
        "lon": 'object',
        "x_pix": 'object',
        "y_pix": 'object',
        "name": 'string',
        "source": 'string'
    }, parse_dates=['datetime'])
df_data.drop(columns="Unnamed: 0", inplace=True)

In [3]:
#df_data#[(df_data.datetime > datetime(2020, 1, 1))]
#df_data[df_data.path.str.contains('20200917_1830')]

In [14]:
gruppi_date = get_gruppi_date(df_data)

In [24]:
d = gruppi_date[0]
lf = list(d.groupby("datetime"))
f = lf[0]
pd.set_option('display.max_columns', 500)
display(f[1])

,path,datetime,tile_offset_x,tile_offset_y,label,lat,lon,x_pix,y_pix,name,source,delta,new_group,gruppo
0,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,0,0,0,[],[],[],[],[],[],NaT,False,0
11,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,1065,196,0,[],[],[],[],[],[],0 days,False,0
10,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,852,196,0,[],[],[],[],[],[],0 days,False,0
9,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,639,196,0,[],[],[],[],[],[],0 days,False,0
7,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,213,196,0,[],[],[],[],[],[],0 days,False,0
6,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,0,196,0,[],[],[],[],[],[],0 days,False,0
8,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,426,196,0,[],[],[],[],[],[],0 days,False,0
4,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,852,0,0,[],[],[],[],[],[],0 days,False,0
3,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,639,0,0,[],[],[],[],[],[],0 days,False,0
2,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,426,0,0,[],[],[],[],[],[],0 days,False,0


### verifico i gruppi di periodi contigui

In [ ]:
lista_date = []
lungh_gruppi = []
date_inizio_fine_gruppi = []
for df in gruppi_date:
    group_datetime = list(df.groupby("datetime"))
    for p, gd in group_datetime:
        lista_date.append(len(gd))
        #print(p)
    lungh_gruppi.append(len(df)/len(gd)) # divido per le 20 tile di ciascun istante temporale
    start_date = df.datetime.iloc[0]
    end_date = df.datetime.iloc[-1]
    date_inizio_fine_gruppi.append((start_date, end_date))


In [17]:
np.array(lungh_gruppi)/288, set(lista_date) # abbiamo 20 tile per ogni data

(array([ 10.00347222,  10.00347222,   8.00347222,   8.00347222,
          4.98611111,   2.93402778,   5.37847222,  54.65625   ,
        277.73611111,   4.00347222,   5.37847222,   8.97916667,
          0.81597222,   8.02083333]),
 {12})

In [16]:
date_inizio_fine_gruppi

[(Timestamp('2011-11-01 00:00:00'), Timestamp('2011-11-11 00:00:00')),
 (Timestamp('2014-11-03 00:00:00'), Timestamp('2014-11-13 00:00:00')),
 (Timestamp('2016-10-26 00:00:00'), Timestamp('2016-11-03 00:00:00')),
 (Timestamp('2017-11-13 00:00:00'), Timestamp('2017-11-21 00:00:00')),
 (Timestamp('2018-09-25 00:00:00'), Timestamp('2018-09-29 23:35:00')),
 (Timestamp('2018-09-30 01:40:00'), Timestamp('2018-10-03 00:00:00')),
 (Timestamp('2020-01-01 00:00:00'), Timestamp('2020-01-06 09:00:00')),
 (Timestamp('2020-02-03 09:05:00'), Timestamp('2020-03-29 00:55:00')),
 (Timestamp('2020-03-29 02:00:00'), Timestamp('2020-12-31 23:55:00')),
 (Timestamp('2021-10-29 00:00:00'), Timestamp('2021-11-02 00:00:00')),
 (Timestamp('2021-11-04 00:00:00'), Timestamp('2021-11-09 09:00:00')),
 (Timestamp('2023-02-24 00:00:00'), Timestamp('2023-03-05 00:00:00')),
 (Timestamp('2023-09-03 00:00:00'), Timestamp('2023-09-03 19:30:00')),
 (Timestamp('2023-09-03 23:25:00'), Timestamp('2023-09-12 00:00:00'))]

### Sembra ben formato -> procedo a creare i video, per ogni blocco temporale

##### prove

In [10]:
df = gruppi_date[0]
df.shape[0]

34572

In [ ]:
df_offsets_groups = group_df_by_offsets(df)
df_videos = create_tile_videos(df_offsets_groups)
df_videos

In [ ]:
#grouped = df.groupby(["tile_offset_x", "tile_offset_y"], group_keys=False)  # ottengo 20 gruppi  perché tante sono le tiles

In [ ]:
df_offsets_groups = group_df_by_offsets(df)
df_videos = create_tile_videos(df_offsets_groups)

m_cicloni = df_videos.label == 1
m_non_cicloni = df_videos.label == 0

df_cicloni = df_videos[m_cicloni]
df_non_cicloni = df_videos[m_non_cicloni]

print(len(df_cicloni), len(df_non_cicloni), len(df_videos))

1350 48640 49990


In [45]:
df_0_balanced = df_non_cicloni.sample(len(df_cicloni))
len(df_0_balanced)

1350

In [46]:
# riduco un pò a piacere i dataframe
df_cicloni = df_cicloni[:500]
df_0_balanced = df_0_balanced[:500]

In [47]:
df_cicloni

,video_id,tile_offset_x,tile_offset_y,path,label,start_time,end_time,orig_paths
3105,3105,0,0,17-09-2020_1830_0_0,1,2020-09-17 17:10:00,2020-09-17 18:30:00,"[../fromgcloud/airmass_rgb_20200917_1710.png, ..."
3106,3106,0,0,17-09-2020_1950_0_0,1,2020-09-17 18:30:00,2020-09-17 19:50:00,"[../fromgcloud/airmass_rgb_20200917_1830.png, ..."
3107,3107,0,0,17-09-2020_2110_0_0,1,2020-09-17 19:50:00,2020-09-17 21:10:00,"[../fromgcloud/airmass_rgb_20200917_1950.png, ..."
3108,3108,0,0,17-09-2020_2230_0_0,1,2020-09-17 21:10:00,2020-09-17 22:30:00,"[../fromgcloud/airmass_rgb_20200917_2110.png, ..."
3109,3109,0,0,17-09-2020_2350_0_0,1,2020-09-17 22:30:00,2020-09-17 23:50:00,"[../fromgcloud/airmass_rgb_20200917_2230.png, ..."
...,...,...,...,...,...,...,...,...
24480,24480,448,0,03-12-2020_0925_448_0,1,2020-12-03 08:05:00,2020-12-03 09:25:00,"[../fromgcloud/airmass_rgb_20201203_0805.png, ..."
24481,24481,448,0,03-12-2020_1045_448_0,1,2020-12-03 09:25:00,2020-12-03 10:45:00,"[../fromgcloud/airmass_rgb_20201203_0925.png, ..."
24482,24482,448,0,03-12-2020_1205_448_0,1,2020-12-03 10:45:00,2020-12-03 12:05:00,"[../fromgcloud/airmass_rgb_20201203_1045.png, ..."
24483,24483,448,0,03-12-2020_1325_448_0,1,2020-12-03 12:05:00,2020-12-03 13:25:00,"[../fromgcloud/airmass_rgb_20201203_1205.png, ..."


In [48]:
# creazione delle folder per i video
create_and_save_tile_from_complete_df(df_0_balanced, output_dir)
create_and_save_tile_from_complete_df(df_cicloni, output_dir)

In [49]:
#df_creati_2011 = pd.concat([df_cicloni, df_0_balanced])
#df_creati_2014 = pd.concat([df_cicloni, df_0_balanced])
#df_creati_2016 = pd.concat([df_cicloni, df_0_balanced])
#df_creati_2017 = pd.concat([df_cicloni, df_0_balanced])
#df_creati_2020_1 = pd.concat([df_cicloni, df_0_balanced])
#df_creati_2020_2 = pd.concat([df_cicloni, df_0_balanced])
df_creati_2020_3 = pd.concat([df_cicloni, df_0_balanced])

##### funzioni caricate 

In [32]:
from build_dataset import balance_time_group

In [ ]:
diz = {}
for i, df in enumerate(gruppi_date):
    if i == 6 or i == 7:
        df_period = balance_time_group(df, output_dir)
        period = date_inizio_fine_gruppi[i]
        print(f"{len(df_period)} video per il periodo dal {period[0]} al {period[1]} \n")
        diz[period] = df_period

In [9]:
df_list = list(diz.items())
l = [(df.shape[0], p) for p,df in df_list]
lunghezze = [len for len, p in l]

In [ ]:
# come prendo e suddivido la lista dei df in train e test
train_df_list_len = 6
test_df_list_len = 2

print(sum(lunghezze[:train_df_list_len]))
print(sum(lunghezze[train_df_list_len:train_df_list_len+test_df_list_len]))

1252
702


In [50]:
#df_train = pd.concat([df_creati_2011,df_creati_2014,df_creati_2016,df_creati_2017,#df_creati_2020_1,#df_creati_2020_2])
df_train = pd.concat([df for p, df in df_list][:train_df_list_len])
df_train

,video_id,tile_offset_x,tile_offset_y,path,label,start_time,end_time,orig_paths
436,436,213,0,05-11-2011_0635_213_0,1,2011-11-05 05:20:00,2011-11-05 06:35:00,"[../fromgcloud/airmass_rgb_20111105_0520.png, ..."
437,437,213,0,05-11-2011_0755_213_0,1,2011-11-05 06:40:00,2011-11-05 07:55:00,"[../fromgcloud/airmass_rgb_20111105_0640.png, ..."
438,438,213,0,05-11-2011_0915_213_0,1,2011-11-05 08:00:00,2011-11-05 09:15:00,"[../fromgcloud/airmass_rgb_20111105_0800.png, ..."
439,439,213,0,05-11-2011_1035_213_0,1,2011-11-05 09:20:00,2011-11-05 10:35:00,"[../fromgcloud/airmass_rgb_20111105_0920.png, ..."
440,440,213,0,05-11-2011_1155_213_0,1,2011-11-05 10:40:00,2011-11-05 11:55:00,"[../fromgcloud/airmass_rgb_20111105_1040.png, ..."
...,...,...,...,...,...,...,...,...
262,262,426,196,30-09-2018_0535_426_196,0,2018-09-30 04:20:00,2018-09-30 05:35:00,"[../fromgcloud/airmass_rgb_20180930_0420.png, ..."
509,509,852,196,02-10-2018_0935_852_196,0,2018-10-02 08:20:00,2018-10-02 09:35:00,"[../fromgcloud/airmass_rgb_20181002_0820.png, ..."
177,177,213,196,01-10-2018_0655_213_196,0,2018-10-01 05:40:00,2018-10-01 06:55:00,"[../fromgcloud/airmass_rgb_20181001_0540.png, ..."
49,49,0,0,02-10-2018_2015_0_0,0,2018-10-02 19:00:00,2018-10-02 20:15:00,"[../fromgcloud/airmass_rgb_20181002_1900.png, ..."


In [51]:
df_test = pd.concat([df for p, df in df_list][train_df_list_len:train_df_list_len + test_df_list_len])
df_test

,video_id,tile_offset_x,tile_offset_y,path,label,start_time,end_time,orig_paths
841,841,852,0,05-01-2020_0235_852_0,1,2020-01-05 01:20:00,2020-01-05 02:35:00,"[../fromgcloud/airmass_rgb_20200105_0120.png, ..."
842,842,852,0,05-01-2020_0355_852_0,1,2020-01-05 02:40:00,2020-01-05 03:55:00,"[../fromgcloud/airmass_rgb_20200105_0240.png, ..."
843,843,852,0,05-01-2020_0515_852_0,1,2020-01-05 04:00:00,2020-01-05 05:15:00,"[../fromgcloud/airmass_rgb_20200105_0400.png, ..."
844,844,852,0,05-01-2020_0635_852_0,1,2020-01-05 05:20:00,2020-01-05 06:35:00,"[../fromgcloud/airmass_rgb_20200105_0520.png, ..."
845,845,852,0,05-01-2020_0755_852_0,1,2020-01-05 06:40:00,2020-01-05 07:55:00,"[../fromgcloud/airmass_rgb_20200105_0640.png, ..."
...,...,...,...,...,...,...,...,...
1099,1099,0,196,09-02-2020_2100_0_196,0,2020-02-09 19:45:00,2020-02-09 21:00:00,"[../fromgcloud/airmass_rgb_20200209_1945.png, ..."
2184,2184,213,0,15-02-2020_1300_213_0,0,2020-02-15 11:45:00,2020-02-15 13:00:00,"[../fromgcloud/airmass_rgb_20200215_1145.png, ..."
2587,2587,213,0,08-03-2020_2230_213_0,0,2020-03-08 21:15:00,2020-03-08 22:30:00,"[../fromgcloud/airmass_rgb_20200308_2115.png, ..."
9011,9011,852,196,12-02-2020_1300_852_196,0,2020-02-12 11:45:00,2020-02-12 13:00:00,"[../fromgcloud/airmass_rgb_20200212_1145.png, ..."


In [51]:
df_test = df_creati_2020_3

### Creazione csv per il training

In [55]:
df_dataset_csv_train = create_final_df_csv(df_train, output_dir)
df_dataset_csv_test = create_final_df_csv(df_test, output_dir)

In [ ]:
df_dataset_csv_train.shape, df_dataset_csv_test.shape,

((1252, 4), (702, 4))

In [59]:
df_dataset_csv_train.to_csv("./train_supervised.csv", index=False)
df_dataset_csv_test.to_csv("./val_supervised.csv", index=False)

# TEST con le stesse funzioni in classe BuildDataset

In [2]:
from dataset.data_manager import BuildDataset

In [7]:
sup_data_train = BuildDataset(type='SUPERVISED', master_df_path="all_data_full_tiles.csv")

In [8]:
sup_data_train.load_master_df()

In [9]:
sup_data_train.get_sequential_periods()
sup_data_train.print_sequential_periods()

1:	2011-11-01 00:00 → 2011-11-11 00:00 - Δ 10 days 00:00:00
2:	2014-11-03 00:00 → 2014-11-13 00:00 - Δ 10 days 00:00:00
3:	2016-10-26 00:00 → 2016-11-03 00:00 - Δ 8 days 00:00:00
4:	2017-11-13 00:00 → 2017-11-21 00:00 - Δ 8 days 00:00:00
5:	2018-09-25 00:00 → 2018-09-29 23:35 - Δ 4 days 23:35:00
6:	2018-09-30 01:40 → 2018-10-03 00:00 - Δ 2 days 22:20:00
7:	2020-01-01 00:00 → 2020-01-06 09:00 - Δ 5 days 09:00:00
8:	2020-02-03 09:05 → 2020-03-29 00:55 - Δ 54 days 15:50:00
9:	2020-03-29 02:00 → 2020-12-31 23:55 - Δ 277 days 21:55:00
10:	2021-10-29 00:00 → 2021-11-02 00:00 - Δ 4 days 00:00:00
11:	2021-11-04 00:00 → 2021-11-09 09:00 - Δ 5 days 09:00:00
12:	2023-02-24 00:00 → 2023-03-05 00:00 - Δ 9 days 00:00:00
13:	2023-09-03 00:00 → 2023-09-03 19:30 - Δ 0 days 19:30:00
14:	2023-09-03 23:25 → 2023-09-12 00:00 - Δ 8 days 00:35:00


In [10]:
sup_data_train.make_df_video(output_dir, idxs=[1,2,3,4,5,6,7,8], is_to_balance=True)

1)  ->
Num video CON cicloni: 141, Num video SENZA cicloni: 2019	Totale video tiles: 2160		Bilanciamento video...
 video senza cicloni tenuti: 141
Creazione delle folder per i 141 video...	Salvati 0 file - Erano già presenti 2256 file - File totali 2256
Creazione delle folder per i 141 video...	Salvati 0 file - Erano già presenti 2256 file - File totali 2256
282 video per il periodo (effettivo) da 2011-11-01 01:20:00 a 2011-11-10 23:55:00

2)  ->
Num video CON cicloni: 143, Num video SENZA cicloni: 2017	Totale video tiles: 2160		Bilanciamento video...
 video senza cicloni tenuti: 143
Creazione delle folder per i 143 video...	Salvati 0 file - Erano già presenti 2288 file - File totali 2288
Creazione delle folder per i 143 video...	Salvati 0 file - Erano già presenti 2288 file - File totali 2288
286 video per il periodo (effettivo) da 2014-11-03 00:00:00 a 2014-11-12 22:35:00

3)  ->
Num video CON cicloni: 106, Num video SENZA cicloni: 1622	Totale video tiles: 1728		Bilanciamento video..

In [11]:
sup_data_train.df_video.label.sum(), sup_data_train.df_video.shape[0],

(977, 1954)

In [12]:
sup_data_train.create_final_df_csv(output_dir, "train_dataset_1954.csv")

In [13]:
sup_data_test = BuildDataset(type='SUPERVISED', master_df_path="all_data_full_tiles.csv")
sup_data_test.load_master_df()
sup_data_test.make_df_video(output_dir, idxs=[9], is_to_balance=True)

9)  ->
Num video CON cicloni: 1401, Num video SENZA cicloni: 58587	Totale video tiles: 59988		Bilanciamento video...
 video senza cicloni tenuti: 1401
Creazione delle folder per i 1401 video...	Salvati 0 file - Erano già presenti 22416 file - File totali 22416
Creazione delle folder per i 1401 video...	Salvati 0 file - Erano già presenti 22416 file - File totali 22416
2802 video per il periodo (effettivo) da 2020-03-29 04:40:00 a 2020-12-31 22:15:00



In [15]:
sup_data_test.df_video.label.sum(), sup_data_test.df_video.shape[0],

(1401, 2802)

In [16]:
sup_data_test.create_final_df_csv(output_dir, "test_dataset_2802.csv")

In [18]:
24*16*3

1152

# Altro

### Altre prove di salvataggio csv train

In [38]:
pd.set_option('display.max_rows', 500)

In [ ]:
df_test = pd.concat([df for p, df in df_list])
df_dataset_csv_test = create_final_df_csv(df_test, output_dir)
df_dataset_csv_test.to_csv("./val_supervised_oggi2.csv", index=False)


In [53]:
df_test[df_test.label==1].sort_values(by='start_time').head()

,video_id,tile_offset_x,tile_offset_y,path,label,start_time,end_time,orig_paths
873,873,852,196,01-01-2020_1315_852_196,1,2020-01-01 12:00:00,2020-01-01 13:15:00,"[../fromgcloud/airmass_rgb_20200101_1200.png, ..."
874,874,852,196,01-01-2020_1435_852_196,1,2020-01-01 13:20:00,2020-01-01 14:35:00,"[../fromgcloud/airmass_rgb_20200101_1320.png, ..."
875,875,852,196,01-01-2020_1555_852_196,1,2020-01-01 14:40:00,2020-01-01 15:55:00,"[../fromgcloud/airmass_rgb_20200101_1440.png, ..."
876,876,852,196,01-01-2020_1715_852_196,1,2020-01-01 16:00:00,2020-01-01 17:15:00,"[../fromgcloud/airmass_rgb_20200101_1600.png, ..."
877,877,852,196,01-01-2020_1835_852_196,1,2020-01-01 17:20:00,2020-01-01 18:35:00,"[../fromgcloud/airmass_rgb_20200101_1720.png, ..."


In [51]:
#df_videos[(df_videos.label == 1) & (df_videos.start_time > datetime(2020, 1, 1)) & (df_videos.end_time < datetime(2020, 12, 31))].sort_values(by='start_time').iloc[:500]

In [26]:
#df_offsets_groups = group_df_by_offsets(df_data)
#df_videos = create_tile_videos_relabeling(df_offsets_groups)
df_videos

,video_id,tile_offset_x,tile_offset_y,path,label,start_time,end_time,orig_paths
0,0,0,0,01-11-2011_0115_0_0,0,2011-11-01 00:00:00,2011-11-01 01:15:00,"[../fromgcloud/airmass_rgb_20111101_0000.png, ..."
1,1,0,0,01-11-2011_0235_0_0,0,2011-11-01 01:20:00,2011-11-01 02:35:00,"[../fromgcloud/airmass_rgb_20111101_0120.png, ..."
2,2,0,0,01-11-2011_0355_0_0,0,2011-11-01 02:40:00,2011-11-01 03:55:00,"[../fromgcloud/airmass_rgb_20111101_0240.png, ..."
3,3,0,0,01-11-2011_0515_0_0,0,2011-11-01 04:00:00,2011-11-01 05:15:00,"[../fromgcloud/airmass_rgb_20111101_0400.png, ..."
4,4,0,0,01-11-2011_0635_0_0,0,2011-11-01 05:20:00,2011-11-01 06:35:00,"[../fromgcloud/airmass_rgb_20111101_0520.png, ..."
...,...,...,...,...,...,...,...,...
88315,88315,1065,196,11-09-2023_1820_1065_196,0,2023-09-11 17:05:00,2023-09-11 18:20:00,"[../fromgcloud/airmass_rgb_20230911_1705.png, ..."
88316,88316,1065,196,11-09-2023_1940_1065_196,0,2023-09-11 18:25:00,2023-09-11 19:40:00,"[../fromgcloud/airmass_rgb_20230911_1825.png, ..."
88317,88317,1065,196,11-09-2023_2100_1065_196,0,2023-09-11 19:45:00,2023-09-11 21:00:00,"[../fromgcloud/airmass_rgb_20230911_1945.png, ..."
88318,88318,1065,196,11-09-2023_2220_1065_196,0,2023-09-11 21:05:00,2023-09-11 22:20:00,"[../fromgcloud/airmass_rgb_20230911_2105.png, ..."


In [35]:
#pd.merge(df_test, df_videos, on='path')
pd.merge(df_videos, df_test, on='path')

,video_id_x,tile_offset_x_x,tile_offset_y_x,path,label_x,start_time_x,end_time_x,orig_paths_x,video_id_y,tile_offset_x_y,tile_offset_y_y,label_y,start_time_y,end_time_y,orig_paths_y


In [27]:
df_test['path'] = df_test['path'].astype(str).str.strip()
df_videos['path'] = df_videos['path'].astype(str).str.strip()
df_test['path'].isin(df_videos['path']).value_counts()

path
False    702
Name: count, dtype: int64

##### verifica indietro

In [54]:
val_dataset = pd.read_csv('val_supervised_oggi2.csv')
val_dataset.path = val_dataset.path.str.split('/').str.get(-1)
val_dataset

,path,start,end,label
0,05-01-2020_0235_852_0,1,16,1
1,05-01-2020_0355_852_0,1,16,1
2,05-01-2020_0515_852_0,1,16,1
3,05-01-2020_0635_852_0,1,16,1
4,05-01-2020_0755_852_0,1,16,1
...,...,...,...,...
697,23-02-2020_1540_426_0,1,16,0
698,14-02-2020_1700_639_0,1,16,0
699,03-03-2020_0910_213_0,1,16,0
700,16-02-2020_1140_213_0,1,16,0


In [20]:
from arguments import prepare_finetuning_args, Args
from dataset import build_dataset
from torch.utils.data import DataLoader
from model_analysis import get_path_pred_label, create_df_predictions, get_only_labels
from dataset_labeling_study import create_tile_videos_relabeling

/home/isac/miniconda3/envs/videomae/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:


args = prepare_finetuning_args()

##### prende da val_supervised.csv
dataset_val, _ = build_dataset(is_train=False, test_mode=False, args=args)  

data_loader_val = DataLoader(
    dataset_val,
    batch_size=args.batch_size,
    shuffle=True,         # Per estrarre sample casuali
    num_workers=args.num_workers,
    pin_memory=args.pin_mem,
    drop_last=False
)

get_prediction = False

all_paths, all_preds, all_labels = get_only_labels(data_loader_val)
print(len(all_paths), len(all_preds), len(all_labels))
df_predictions = create_df_predictions(all_paths, all_preds, all_labels)
df_predictions.shape[0]

Number of the class = 2
702 702 702


702

In [22]:
df_p = pd.DataFrame(all_paths, columns=['path'])
df_p.path = df_p.path.str.split('/').str.get(-1)
print(df_p.merge(val_dataset, on='path').shape)

val_dataset.merge(df_predictions, on='path')
# -> fin quì TORNA

(702, 4)


,path,start,end,label,predictions,labels
0,05-01-2020_0235_852_0,1,16,1,0,1
1,05-01-2020_0355_852_0,1,16,1,0,1
2,05-01-2020_0515_852_0,1,16,1,0,1
3,05-01-2020_0635_852_0,1,16,1,0,1
4,05-01-2020_0755_852_0,1,16,1,0,1
...,...,...,...,...,...,...
697,23-02-2020_1540_426_0,1,16,0,0,0
698,14-02-2020_1700_639_0,1,16,0,0,0
699,03-03-2020_0910_213_0,1,16,0,0,0
700,16-02-2020_1140_213_0,1,16,0,0,0


In [62]:
df_data_time_list = [gruppi_date[6], gruppi_date[7]]
df_videos = []
for df_data_time_group in df_data_time_list:
    df_offsets_groups = group_df_by_offsets(df_data_time_group)
    df_video = create_tile_videos_relabeling(df_offsets_groups)
    df_videos.append(df_video)
df_videos = pd.concat(df_videos)

Righe del df_video: 1152
Righe del df_video: 11796


In [24]:
df_data.shape[0]/16

88323.0

In [64]:
df_videos.merge(df_predictions, on='path').sort_values(by='start_time')
#df_videos[df_videos.path == '24-03-2020_1950_213_196']


,video_id,tile_offset_x,tile_offset_y,path,label,start_time,end_time,orig_paths,predictions,labels
73,768,852,0,01-01-2020_0115_852_0,0,2020-01-01 00:00:00,2020-01-01 01:15:00,"[../fromgcloud/airmass_rgb_20200101_0000.png, ...",0,0
5,97,0,196,01-01-2020_0235_0_196,0,2020-01-01 01:20:00,2020-01-01 02:35:00,"[../fromgcloud/airmass_rgb_20200101_0120.png, ...",0,0
56,577,639,0,01-01-2020_0235_639_0,0,2020-01-01 01:20:00,2020-01-01 02:35:00,"[../fromgcloud/airmass_rgb_20200101_0120.png, ...",0,0
45,484,426,196,01-01-2020_0635_426_196,0,2020-01-01 05:20:00,2020-01-01 06:35:00,"[../fromgcloud/airmass_rgb_20200101_0520.png, ...",0,0
143,1060,1065,196,01-01-2020_0635_1065_196,0,2020-01-01 05:20:00,2020-01-01 06:35:00,"[../fromgcloud/airmass_rgb_20200101_0520.png, ...",0,0
...,...,...,...,...,...,...,...,...,...,...
474,5886,426,196,28-03-2020_0910_426_196,0,2020-03-28 07:55:00,2020-03-28 09:10:00,"[../fromgcloud/airmass_rgb_20200328_0755.png, ...",0,0
610,9821,852,196,28-03-2020_1310_852_196,0,2020-03-28 11:55:00,2020-03-28 13:10:00,"[../fromgcloud/airmass_rgb_20200328_1155.png, ...",0,0
627,10804,1065,0,28-03-2020_1310_1065_0,0,2020-03-28 11:55:00,2020-03-28 13:10:00,"[../fromgcloud/airmass_rgb_20200328_1155.png, ...",0,0
701,11788,1065,196,28-03-2020_1430_1065_196,0,2020-03-28 13:15:00,2020-03-28 14:30:00,"[../fromgcloud/airmass_rgb_20200328_1315.png, ...",0,0


### Guardo alcuni video da cancellare dal disco per risparmiare spazio

In [128]:
df_out[df_out['path'].str.contains(r"/part43\d+", regex=True)].label.sum()

0

In [129]:
df_filt = df_out[df_out['path'].apply(os.path.isdir)]
print(f"Numero di cartelle esistenti: {len(df_filt)}, righe totali precedenti: {df_out.shape[0]}")

Numero di cartelle esistenti: 126001, righe totali precedenti: 147200


In [130]:
df_filt[df_filt.label==1].shape[0], df_filt[df_filt.label==0].shape[0], round(df_filt.label.sum()/df_filt.shape[0], 3)

(1512, 124489, 0.012)

In [62]:
def balance_df_by_label(df, label_column="label", random_state=478562):
    """
    Bilancia un DataFrame mantenendo tutte le righe con label=1
    e selezionando (in modo random) lo stesso numero di righe con label=0.
    Restituisce un DataFrame mescolato con uguale quantità di 1 e 0.
    
    Parametri:
    - df: DataFrame originale
    - label_column: il nome della colonna che contiene la label (default: 'label')
    - random_state: per riproducibilità del campionamento casuale
    """
    # Seleziona tutte le righe con label=1
    df_pos = df[df[label_column] == 1]
    # Seleziona tutte le righe con label=0
    df_neg = df[df[label_column] == 0]

    # Numero di righe positive
    num_pos = len(df_pos)
    # Esegui un sample sulle righe negative pari a num_pos
    df_neg_sampled = df_neg.sample(n=num_pos, random_state=random_state)

    # Concatena e mescola
    df_balanced = pd.concat([df_pos, df_neg_sampled])
    df_balanced = df_balanced.sample(frac=1, random_state=random_state).reset_index(drop=True)

    return df_balanced

In [131]:
df_balanced = balance_df_by_label(df_filt)
print(len(df_balanced), df_balanced['label'].value_counts())

3024 label
1    1512
0    1512
Name: count, dtype: int64


In [137]:
df_balanced


,path,start,end,label,start_time,end_time
0,../airmassRGB/supervised/part6762_784_112,1,16,1,2020-12-25 21:45:00,2020-12-25 23:00:00
1,../airmassRGB/supervised/part6102_784_112,1,16,1,2020-11-19 04:50:00,2020-11-19 06:05:00
2,../airmassRGB/supervised/part6101_784_112,1,16,1,2020-11-19 03:30:00,2020-11-19 04:45:00
3,../airmassRGB/supervised/part3246_560_0,1,16,1,2020-06-13 11:20:00,2020-06-13 12:35:00
4,../airmassRGB/supervised/part1925_336_112,1,16,0,2020-04-01 00:05:00,2020-04-01 01:20:00
...,...,...,...,...,...,...
3019,../airmassRGB/supervised/part1653_560_0,1,16,1,2020-03-16 20:25:00,2020-03-16 21:40:00
3020,../airmassRGB/supervised/part1993_896_112,1,16,0,2020-04-04 18:45:00,2020-04-04 20:00:00
3021,../airmassRGB/supervised/part3207_448_112,1,16,1,2020-06-11 07:20:00,2020-06-11 08:35:00
3022,../airmassRGB/supervised/part2788_336_112,1,16,0,2020-05-18 23:05:00,2020-05-19 00:20:00


In [135]:
l = df_balanced.shape[0]
#l = 100
tr_p = int(l * 0.7)
val_p = int(l * 0.9)
print(f"ranges train e validation:  {tr_p} - {val_p}")
df_train = df_balanced[:tr_p]
df_val = df_balanced[tr_p:val_p]
df_test = df_balanced[val_p:]

ranges train e validation:  2116 - 2721


In [136]:
out_csv_label = os.path.join("./", "train_supervised.csv")
df_train.to_csv(out_csv_label, index=False)
out_csv_label = os.path.join("./", "val_supervised.csv")
df_val.to_csv(out_csv_label, index=False)
out_csv_label = os.path.join("./", "test_supervised.csv")
df_test.to_csv(out_csv_label, index=False)

### copia le immagini in sottocartelle .../partN

In [27]:
from build_supervised_dataset_no_tiles import split_into_subfolders_and_track_dates

In [ ]:
#subfolders = split_into_subfolders(sorted_filenames, output_dir)
subfolder_info = split_into_subfolders_and_track_dates(sorted_filenames, output_dir)